In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1718559097767_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Import des packages :

In [2]:
import numpy as np
import pandas as pd
from PIL import Image
import io
import os

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.feature import StandardScaler, PCA
from pyspark.ml import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.preprocessing import LabelEncoder
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Définition des PATH pour charger les images et enregistrer les résultats :

In [3]:
PATH = "s3://oc-p9-bigdata-data"
PATH_Data = PATH+'/Images/Images'
PATH_Result = PATH+'/Images/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://oc-p9-bigdata-data
PATH_Data:   s3://oc-p9-bigdata-data/Images/Images
PATH_Result: s3://oc-p9-bigdata-data/Images/Results

## Traitement des données

### Chargement des données

In [6]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des 5 premières images contenant</u> :
 - le path de l'image
 - la date et heure de sa dernière modification
 - sa longueur
 - son contenu encodé en valeur hexadécimal

In [7]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://oc-p9-bigdat...|2024-06-11 18:29:22|  5365|[FF D8 FF E0 00 1...|
|s3://oc-p9-bigdat...|2024-06-11 18:29:22|  5340|[FF D8 FF E0 00 1...|
|s3://oc-p9-bigdat...|2024-06-11 18:29:22|  5338|[FF D8 FF E0 00 1...|
|s3://oc-p9-bigdat...|2024-06-11 18:29:22|  5331|[FF D8 FF E0 00 1...|
|s3://oc-p9-bigdat...|2024-06-11 18:29:22|  5323|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [8]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-----------------------------------------------------+-----+
|path                                                 |label|
+-----------------------------------------------------+-----+
|s3://oc-p9-bigdata-data/Images/Images/Cocos/3_100.jpg|Cocos|
|s3://oc-p9-bigdata-data/Images/Images/Cocos/8_100.jpg|Cocos|
|s3://oc-p9-bigdata-data/Images/Images/Cocos/7_100.jpg|Cocos|
|s3://oc-p9-bigdata-data/Images/Images/Cocos/1_100.jpg|Cocos|
|s3://oc-p9-bigdata-data/Images/Images/Cocos/9_100.jpg|Cocos|
+-----------------------------------------------------+-----+
only showing top 5 rows

None

### Préparation du modèle :

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 [==============================] - 1s 0us/step

In [10]:
# On enlève la dernière couche du modèle (extration de feature "seulement") :
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Vérif structure du modèle :

new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

Tous les workeurs doivent pouvoir accéder au modèle ainsi qu'à ses poids. <br />
Une bonne pratique consiste à charger le modèle sur le driver puis à diffuser <br />
ensuite les poids aux différents workeurs.

In [12]:
# diffusion des poids du modèle Tensorflow sur les clusters :

brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# On met tout ça sous forme de fonction :

def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Définition du processus de chargement des images et application <br/>de leur featurisation à travers l'utilisation de pandas UDF

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1718265677443_0001/container_1718265677443_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:398: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

### Exécution des actions d'extraction de features

In [15]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nous obtenons une erreur de type de données en voulant réaliser la PCA directement sur la colonne feature créée précédemment.
Nous devons transformer cette dernière en Vecteur UDT.

Par ailleurs, j'aimerais réaliser une PCA retenant 95% de la variance dans les données. Or l'objet PCA de pyspark ne permet pas de le faire directement, je vais donc procéder comme suit : réaliser une PCA avec un nombre de composantes = au nombre des features, puis calculer le nombre de features nécessaires à la conservation des 95% de variances.
Ensuite je réaliserai de nouveau une PCA avec le nombre de composantes obtenu.

In [16]:
# UDF pour convertir les listes de floats en DenseVector
def array_to_vector(array):
    return Vectors.dense(array)

array_to_vector_udf = udf(array_to_vector, VectorUDT())

# Appliquer la conversion
features_df = features_df.withColumn("features", array_to_vector_udf(features_df["features"]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Obtention du nombre de features :

first_row = features_df.first()
num_features = len(first_row["features"])
print(f"Nombre de features : {num_features}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nombre de features : 1280

In [18]:
# On crée un pipeline permettant de scaler les données puis de leur appliquer une
# réduction de dimension par PCA :

# Scaling :
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withStd=True,
    withMean=True
)

# PCA avec un nombre de composantes = nombre de features:

pca = PCA(
    k=num_features,
    inputCol=scaler.getOutputCol(),
    outputCol="pca_features"
)

pipeline = Pipeline(stages=[scaler , pca])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
model = pipeline.fit(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Calculer la variance expliquée
explained_variance = model.stages[-1].explainedVariance.toArray()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Calculer la variance cumulée
cumulative_variance = [sum(explained_variance[:i+1]) for i in range(len(explained_variance))]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Déterminer le nombre de composantes nécessaires pour atteindre le pourcentage de variance souhaité (95%)
target_variance = 0.95
k = next(i for i, cum_var in enumerate(cumulative_variance) if cum_var >= target_variance) + 1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
print(f"Nombre de composantes nécessaires pour expliquer {target_variance*100}% de la variance : {k}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nombre de composantes n?cessaires pour expliquer 95.0% de la variance : 28

In [25]:
# On refait la PCA avec le nombre de composantes k :
pca = PCA(
    k=k,
    inputCol=scaler.getOutputCol(),
    outputCol="pca_features"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
pipeline = Pipeline(stages=[scaler, pca])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
model = pipeline.fit(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
features_df = model.transform(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Enregistrement des données traitées au format "parquet" :

features_df.drop('scaled_features').write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Chargement des données enregistrées et validation du résultat

In [4]:
# On charge les données fraichement enregistrées dans un DataFrame Pandas :

df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Enregistrement du df au format csv sur le bucket S3 :
df.to_csv(PATH_Result, index=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Affichage des 5 premières lignes :

df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                       pca_features
0  s3://oc-p9-bigdata-data/Images/Images/Apricot/...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
1  s3://oc-p9-bigdata-data/Images/Images/Banana/1...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
2  s3://oc-p9-bigdata-data/Images/Images/Apricot/...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
3  s3://oc-p9-bigdata-data/Images/Images/Blueberr...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
4  s3://oc-p9-bigdata-data/Images/Images/Apricot/...  ...  {'type': 1, 'size': None, 'indices': None, 'va...

[5 rows x 4 columns]

In [6]:
# Vérif dimensions colonne features :
df.loc[0, 'features']["values"].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [7]:
# Vérif dimensions colonne pca_features :
df.loc[0, 'pca_features']["values"].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(28,)

On a bien les bonnes dimensions pour nos features.

In [8]:
df.loc[0, 'pca_features']["values"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([-1.64743063e+00, -5.49881393e-02,  1.78766514e+01, -1.78413793e+00,
       -1.87576405e+00, -1.27940283e+00,  1.17737385e+00, -1.23752484e+01,
       -4.89630038e-14, -4.01783641e-14, -3.55384125e-14,  1.30850192e-14,
       -1.78294879e-14,  2.56010491e-14, -1.76404030e-14, -3.37403716e-16,
       -1.03389519e-15, -7.17308157e-16, -3.66151979e+00, -4.93435769e-07,
        1.06154073e+01,  7.46603671e+00, -2.64255670e+00, -2.89224829e+00,
        4.65062731e+00, -6.76841414e+00,  2.79458868e+00,  1.54713432e+00])

In [9]:
df.loc[0, 'path']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://oc-p9-bigdata-data/Images/Images/Apricot/32_100.jpg'